### Acoustic VGGish

In [1]:
import os
import sys
import random

import numpy as np
import torch
import torchaudio.functional as F

from torchvggish.vggish import VGG, VGGish, make_layers
from torchvggish.vggish_input import waveform_to_examples
import torchvggish.mel_features
import torchvggish.vggish_params


### Paramètres globaux à faire varier

In [ ]:
GRID_LENGTH = 200
GRID_CELL_SIZE = 0.01
# GRID_LENGTH = 400
# GRID_CELL_SIZE = 0.005
STEP_FREQUENCY = 88200
NUM_SUBSTEPS = 2
IR_POSITION = [0, 0.7, .2]
SAMPLE_POSITION = [0, -0.7, 0.2]
# IR_POSITION = [0.9, 0, -0.6]
# SAMPLE_POSITION = [0.9, 0, -0.6]
N_STEPS = 8820

# VGGISH
PSEUDO_SAMPLE_RATE = 16000

### 2e tentative

In [6]:
import tensorflow as tf

import numpy as np

from sklearn.model_selection import train_test_split

### Charger le modèle VGGish

In [ ]:
vgg_graph = tf.Graph()
with vgg_graph.as_default():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    vggish_slim.define_vggish_slim(training=True)
    vggish_slim.load_vggish_slim_checkpoint(sess, 'chemin/vers/vggish_model.ckpt')

### Charger les données d'entrée (RIRs) et de sortie (positions reconnues) depuis les fichiers .npy

In [ ]:
input_data = np.load('chemin/vers/fichier_RIRs.npy')
output_data = np.load('chemin/vers/fichier_positions.npy')

### Entrainement / Test

In [ ]:
train_input, test_input, train_output, test_output = train_test_split(input_data, output_data, test_size=0.2, random_state=42)

### Extraction des caractéristiques avec VGGish

In [ ]:
def extract_vggish_features(data):
    features = []
    with vgg_graph.as_default():
        for rir in data:
            spectrogram = vggish_input.waveform_to_examples(rir, vggish_params.SAMPLE_RATE)
            features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
            embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)
            embedding = sess.run(embedding_tensor, feed_dict={features_tensor: spectrogram})
            features.append(embedding)
    return np.array(features)

### Extraction des caractéristiques pour les ensembles d'entraînement et de test

In [ ]:
train_features = extract_vggish_features(train_input)
test_features = extract_vggish_features(test_input)

### Modèle de régression sur les caractéristiques VGGish

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=train_features.shape[1:]),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(output_data.shape[1])
])

model.compile(optimizer='adam', loss='mse')

model.fit(train_features, train_output, epochs=10, batch_size=32, validation_data=(test_features, test_output))

loss = model.evaluate(test_features, test_output)
print("Perte sur les données de test :", loss)